# Torch Image Models (timm)

Библиотека [timm 🛠️[doc]](https://huggingface.co/docs/timm/index) (полное название "Py**T**orch **Im**age **M**odels") — это набор предварительно обученных моделей для задач компьютерного зрения, разработанный для фреймворка PyTorch. TIMM предоставляет широкий спектр моделей, которые можно использовать для задач, таких как классификация изображений, сегментация, обнаружение объектов и другие задачи обработки изображений.

Основные особенности библиотеки:

* **Разнообразие моделей**. Библиотека содержит различные архитектуры нейронных сетей, включая популярные модели, такие как ResNet, EfficientNet, ViT (Vision Transformer) и многие другие.

* **Предварительное обучение**. Модели предварительно обучены на больших датасетах, таких как ImageNet, что позволяет использовать их как основу для различных задач компьютерного зрения с минимальной настройкой.

* **Простота использования**. Удобный интерфейс для загрузки и использования моделей в PyTorch.

* **Гибкость**. Возможность настраивать и адаптировать модели из библиотеки под свои конкретные задачи, добавлять и изменять слои и т. д.

In [ ]:
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:00


In [ ]:
import torch
import random
import numpy as np

# fix random_seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# compute in cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Для начала посмотрим, сколько моделей есть в библиотеке:

In [ ]:
import timm

len(timm.list_models())

1133

Как видим, список довольно большой ([все архитектуры со ссылками на статьи 🛠️[doc]](https://github.com/huggingface/pytorch-image-models#models)). Можем указать параметры, чтобы сузить поиск. Например, посмотрим все предобученные EfficientNet:

In [ ]:
timm.list_models("efficientnet*", pretrained=True)

['efficientnet_b0.ra4_e3600_r224_in1k',
 'efficientnet_b0.ra_in1k',
 'efficientnet_b1.ft_in1k',
 'efficientnet_b1.ra4_e3600_r240_in1k',
 'efficientnet_b1_pruned.in1k',
 'efficientnet_b2.ra_in1k',
 'efficientnet_b2_pruned.in1k',
 'efficientnet_b3.ra2_in1k',
 'efficientnet_b3_pruned.in1k',
 'efficientnet_b4.ra2_in1k',
 'efficientnet_b5.sw_in12k',
 'efficientnet_b5.sw_in12k_ft_in1k',
 'efficientnet_el.ra_in1k',
 'efficientnet_el_pruned.in1k',
 'efficientnet_em.ra2_in1k',
 'efficientnet_es.ra_in1k',
 'efficientnet_es_pruned.in1k',
 'efficientnet_lite0.ra_in1k',
 'efficientnetv2_rw_m.agc_in1k',
 'efficientnetv2_rw_s.ra2_in1k',
 'efficientnetv2_rw_t.ra2_in1k']

Моделей много,чтобы выбрать нужную можно ознакомиться с их качеством в этой [таблице 🛠️[doc]](https://github.com/huggingface/pytorch-image-models/blob/main/results/results-imagenet.csv).

Загрузим одну из моделей:

In [ ]:
from IPython.display import clear_output

model_name = "efficientnet_lite0.ra_in1k"
pretrained_model = timm.create_model(model_name, pretrained=True)
clear_output()
print(pretrained_model)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): ReLU6(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU6(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
    )
    (1): Sequent

Если мы захотим применить модель в своей задаче, то, вероятно, нам потребуется заменить количество нейронов в последнем слое. Например, в случае с CIFAR-10 нам потребуется 10 выходов вместо 1000. Сделать это можно сразу при загрузке модели:

In [ ]:
pretrained_model = timm.create_model(model_name, pretrained=True, num_classes=10)
pretrained_model.classifier

Linear(in_features=1280, out_features=10, bias=True)

Раньше мы делали это вручную:

In [ ]:
import torch.nn as nn

pretrained_model.classifier = nn.Linear(1280, 10)

И эта возможность сохраняется. Мы по-прежнему можем изменить любой слой или блок слоев. Посмотрим высокоуровневую структуру сети.

In [ ]:
layers = dict(pretrained_model.named_children())

print(f"{layers.keys() = }")

layers.keys() = dict_keys(['conv_stem', 'bn1', 'blocks', 'conv_head', 'bn2', 'global_pool', 'classifier'])


In [ ]:
pretrained_model.conv_head

Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)

Изменим параметры, создав новый слой вместо старого:

In [ ]:
from copy import deepcopy

modified_model = deepcopy(pretrained_model)
modified_model.conv_head = nn.Conv2d(
    320, 1280, kernel_size=(3, 3), stride=(1, 1), bias=False
)

**Важно:** следите за совместимостью слоев и будьте осторожны, заменяя крупные блоки модели с именованными атрибутами на `nn.Sequential`. Вы можете изменить логику модели, прописанную в функции `forward`. Например, если вы заменете BasicBlock ResNet на `nn.Sequential` с тем же набором слоев, вы потеряете residual connection.

Проверим, что сеть изменилась:

In [ ]:
modified_model.conv_head

Conv2d(320, 1280, kernel_size=(3, 3), stride=(1, 1), bias=False)

Но при этом работает корректно:

In [ ]:
x = torch.rand(1, 3, 224, 224)
out = modified_model(x)
print(f"{out.shape = }")

out.shape = torch.Size([1, 10])


В timm сеть разделена (условно) на две части: `feature_extractor` и `classifier`. Это позволяет использовать более гибкие методы работы. Получим классификационную часть методом `get_classifier()` (в нашем случае это один слой) и посмотрим, какие параметры на вход ожидает слой:

In [ ]:
num_in_features = pretrained_model.get_classifier().in_features
num_in_features

1280

Теперь заменим слой на собственный классификационный блок:

In [ ]:
pretrained_model.classifier = nn.Sequential(
    nn.Linear(in_features=num_in_features, out_features=512, bias=False),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.4),
    nn.Linear(in_features=512, out_features=10, bias=False),
)

In [ ]:
pretrained_model.classifier

Sequential(
  (0): Linear(in_features=1280, out_features=512, bias=False)
  (1): ReLU()
  (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.4, inplace=False)
  (4): Linear(in_features=512, out_features=10, bias=False)
)

Мы также можем изменить количество входных каналов (параметр `in_chans`), и timm адаптирует слой для использования тензора с нужной размерностью, перераспределяя веса:

In [ ]:
pretrained_model_2 = timm.create_model(
    model_name, pretrained=True, num_classes=10, in_chans=8
)
x = torch.rand(1, 8, 224, 224)
pretrained_model_2(x).shape

torch.Size([1, 10])

## Custom feature extractor

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.2/L08/out/feature_extraction_backbone.png"  width="650"></center>

Разберемся теперь с `feature_extractor`. Используя методы из `torchvision.models.feature_extraction`, мы можем посмотреть граф модели с названиями элементов и получить `output` любого элемента. Тут есть некоторые особенности, с которыми можно ознакомиться в [документации 🛠️[doc]](https://pytorch.org/blog/FX-feature-extraction-torchvision/), но в общем случае это работает так:

* cтроим граф, выбираем, откуда хотим получить `output`;
* cоздаем `feature_extractor` с выбранным элементом.

Построим граф и посмотрим, как он выглядит:

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names

get_graph_node_names(pretrained_model)[0]

['x',
 'conv_stem',
 'bn1.getattr',
 'bn1.eq',
 'bn1.getattr_1',
 'bn1._assert',
 'bn1.bn1_weight',
 'bn1.bn1_bias',
 'bn1.batch_norm',
 'bn1.drop',
 'bn1.act',
 'blocks.0.0.conv_dw',
 'blocks.0.0.bn1.getattr',
 'blocks.0.0.bn1.eq',
 'blocks.0.0.bn1.getattr_1',
 'blocks.0.0.bn1._assert',
 'blocks.0.0.bn1.blocks_0_0_bn1_weight',
 'blocks.0.0.bn1.blocks_0_0_bn1_bias',
 'blocks.0.0.bn1.batch_norm',
 'blocks.0.0.bn1.drop',
 'blocks.0.0.bn1.act',
 'blocks.0.0.aa',
 'blocks.0.0.se',
 'blocks.0.0.conv_pw',
 'blocks.0.0.bn2.getattr',
 'blocks.0.0.bn2.eq',
 'blocks.0.0.bn2.getattr_1',
 'blocks.0.0.bn2._assert',
 'blocks.0.0.bn2.blocks_0_0_bn2_weight',
 'blocks.0.0.bn2.blocks_0_0_bn2_bias',
 'blocks.0.0.bn2.batch_norm',
 'blocks.0.0.bn2.drop',
 'blocks.0.0.bn2.act',
 'blocks.1.0.conv_pw',
 'blocks.1.0.bn1.getattr',
 'blocks.1.0.bn1.eq',
 'blocks.1.0.bn1.getattr_1',
 'blocks.1.0.bn1._assert',
 'blocks.1.0.bn1.blocks_1_0_bn1_weight',
 'blocks.1.0.bn1.blocks_1_0_bn1_bias',
 'blocks.1.0.bn1.batch_no

Для демонстрации возьмем `output` с первого слоя в нашем классификационном блоке:

In [ ]:
from torchvision.models.feature_extraction import create_feature_extractor

features = {"classifier.0": "out"}
custom_fe = create_feature_extractor(pretrained_model, return_nodes=features)
custom_fe

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): Module(
    (drop): Identity()
    (act): ReLU6(inplace=True)
  )
  (blocks): Module(
    (0): Module(
      (0): Module(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): Module(
          (drop): Identity()
          (act): ReLU6(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): Module(
          (drop): Identity()
          (act): Identity()
        )
      )
    )
    (1): Module(
      (0): Module(
        (conv_pw): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): Module(
          (drop): Identity()
          (act): ReLU6(inplace=True)
        )
        (conv_dw): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=96, bias=Fals

In [ ]:
x = torch.rand(1, 3, 224, 224)
custom_fe(x)["out"].shape

torch.Size([1, 512])

Другой, более верхнеуровневый способ — это использовать модель в режиме `features_only`. Загрузим модель:



In [ ]:
fe_model = timm.create_model(model_name, pretrained=True, features_only=True)
clear_output()

Посмотрим параметры блоков:

In [ ]:
list(fe_model.feature_info)

[{'stage': 1, 'reduction': 2, 'module': 'blocks.0', 'num_chs': 16, 'index': 0},
 {'stage': 2, 'reduction': 4, 'module': 'blocks.1', 'num_chs': 24, 'index': 1},
 {'stage': 3, 'reduction': 8, 'module': 'blocks.2', 'num_chs': 40, 'index': 2},
 {'stage': 5,
  'reduction': 16,
  'module': 'blocks.4',
  'num_chs': 112,
  'index': 3},
 {'stage': 7,
  'reduction': 32,
  'module': 'blocks.6',
  'num_chs': 320,
  'index': 4}]

Получим все карты признаков из этих блоков:

In [ ]:
out = fe_model(x)
for output in out:
    print(output.shape)

torch.Size([1, 16, 112, 112])
torch.Size([1, 24, 56, 56])
torch.Size([1, 40, 28, 28])
torch.Size([1, 112, 14, 14])
torch.Size([1, 320, 7, 7])


При использовании предобученной модели важно знать, как и на каких данных она обучалась. В частности нас могут интересовать примененные аугментации (`transform`).

В timm каждая модель содержит описание своей конфигурации `default_cfg` (без обучения) и `pretrained_cfg` (предобученная). Посмотрим на `pretrained_cfg`:

In [ ]:
pretrained_model.pretrained_cfg

{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_lite0_ra-37913777.pth',
 'hf_hub_id': 'timm/efficientnet_lite0.ra_in1k',
 'architecture': 'efficientnet_lite0',
 'tag': 'ra_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'fixed_input_size': False,
 'interpolation': 'bicubic',
 'crop_pct': 0.875,
 'crop_mode': 'center',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'num_classes': 1000,
 'pool_size': (7, 7),
 'first_conv': 'conv_stem',
 'classifier': 'classifier'}

Создадим `transform`:

In [ ]:
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

transform = create_transform(
    **resolve_data_config(pretrained_model.pretrained_cfg, model=pretrained_model)
)
transform

Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

Мы кратко познакомились с библиотекой timm. Возможностей у нее гораздо больше, но в рамках лекции мы никак не успеем с ними познакомиться.

Рекомендуем для самостоятельного изучения:
* [[blog] ✏️ Fine-Tuning Image Classifiers with PyTorch and the timm library for Beginners](https://christianjmills.com/posts/pytorch-train-image-classifier-timm-hf-tutorial/)
* [[doc] 🛠️ Документация библиотеки](https://github.com/huggingface/pytorch-image-models#introduction)